# TỐI ƯU HÓA THÔNG SỐ MÔ HÌNH
Bây giờ chúng tôi đã có mô hình và dữ liệu, đã đến lúc đào tạo, xác thực và kiểm tra mô hình của mình bằng cách tối ưu hóa các thông số của nó trên dữ liệu của chúng tôi. Đào tạo một mô hình là một quá trình lặp đi lặp lại; trong mỗi lần lặp (được gọi là một epoch), mô hình đưa ra phỏng đoán về kết quả đầu ra, tính toán lỗi trong dự đoán của nó (tổn thất), thu thập các đạo hàm của lỗi đối với các tham số của nó (như chúng ta đã thấy trong [phần trước](https://pytorch.org/tutorials/beginner/basics/autograd_tutorial.html)) và tối ưu hóa các tham số này bằng cách sử dụng gradient descent. Để có hướng dẫn chi tiết hơn về quy trình này, hãy xem video này về nhân giống ngược từ [3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Mã tiên quyết
Chúng tôi tải mã từ các phần trước trên Datasets & DataLoaders và Build Model.

In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Siêu tham số (Hyperparameters)
Hyperparameters là các tham số có thể điều chỉnh cho phép bạn kiểm soát quá trình tối ưu hóa mô hình. Các giá trị siêu tham số khác nhau có thể ảnh hưởng đến việc đào tạo mô hình và tốc độ hội tụ (đọc thêm về điều chỉnh siêu tham số)

Chúng tôi xác định các siêu tham số sau để đào tạo:

- **Number of Epochs** - số lần lặp lại trên tập dữ liệu
- **Batch Size** - số lượng mẫu dữ liệu được truyền qua mạng trước khi các tham số được cập nhật
- **Learning Rate** - mức độ cập nhật các thông số mô hình tại batch/epoch. Giá trị nhỏ hơn mang lại tốc độ học tập chậm, trong khi giá trị lớn có thể dẫn đến hành vi không thể đoán trước trong quá trình đào tạo.

In [14]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Vòng lặp tối ưu hóa
Khi chúng tôi đặt siêu tham số, chúng tôi có thể đào tạo và tối ưu hóa mô hình của mình bằng một vòng lặp tối ưu hóa. Mỗi lần lặp của vòng lặp tối ưu hóa được gọi là một epoch.

Mỗi epoch bao gồm hai phần chính:
- The Train Loop - lặp qua tập dữ liệu huấn luyện và cố gắng hội tụ đến các tham số tối ưu.
- The Validation/Test Loop - lặp lại tập dữ liệu thử nghiệm để kiểm tra xem hiệu suất của mô hình có được cải thiện hay không.

Chúng ta hãy làm quen với một số khái niệm được sử dụng trong vòng đào tạo. Hãy tiếp tục để xem Triển khai đầy đủ của vòng lặp tối ưu hóa.

## Loss Function
Khi được trình bày với một số dữ liệu đào tạo, mạng chưa được đào tạo của chúng tôi có thể không đưa ra câu trả lời chính xác. Hàm tổn thất đo lường mức độ không giống nhau của kết quả thu được với giá trị mục tiêu và đó là hàm tổn thất mà chúng tôi muốn giảm thiểu trong quá trình đào tạo. Để tính toán tổn thất, chúng tôi đưa ra dự đoán bằng cách sử dụng đầu vào của mẫu dữ liệu đã cho của chúng tôi và so sánh nó với giá trị nhãn dữ liệu thực.

Các hàm mất mát phổ biến bao gồm `nn.MSELoss` (Mean Square Error) cho các tác vụ hồi quy và `nn.NLLLoss` (Negative Log Likelihood) để phân loại. `nn.CrossEntropyLoss` kết hợp `nn.LogSoftmax` và `nn.NLLLoss`.

Chúng tôi chuyển nhật ký đầu ra của mô hình của chúng tôi tới `nn.CrossEntropyLoss`, sẽ chuẩn hóa nhật ký và tính toán lỗi dự đoán.

In [15]:
loss_fn = nn.CrossEntropyLoss()

## Optimizer (Trình tối ưu hóa)
Tối ưu hóa là quá trình điều chỉnh các tham số của mô hình để giảm lỗi mô hình trong mỗi bước huấn luyện. Các thuật toán tối ưu hóa xác định cách quá trình này được thực hiện (trong ví dụ này, chúng tôi sử dụng Stochastic Gradient Descent). Tất cả logic tối ưu hóa được gói gọn trong đối tượng trình `optimizer`. Ở đây, chúng tôi sử dụng trình tối ưu hóa SGD; Ngoài ra, có nhiều trình tối ưu hóa khác nhau có sẵn trong PyTorch như ADAM và RMSProp, hoạt động tốt hơn cho các loại mô hình và dữ liệu khác nhau.

Chúng tôi khởi chạy trình tối ưu hóa bằng cách đăng ký các tham số của mô hình cần được đào tạo và chuyển vào siêu thông số tốc độ học.

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Bên trong vòng lặp đào tạo, việc tối ưu hóa diễn ra theo ba bước:
- Gọi `optimizer.zero_grad()` để đặt lại gradient của các thông số mô hình. Gradients theo mặc định cộng lên; để ngăn việc đếm hai lần, chúng tôi rõ ràng bằng không chúng ở mỗi lần lặp.
- Đảo ngược sự mất mát dự đoán bằng lời gọi `loss.backwards ()`. PyTorch ghi nhận mức độ lỗ w.r.t. mỗi tham số.
- Khi chúng ta đã có các gradient của mình, chúng ta gọi `Optimizer.step()` để điều chỉnh các tham số theo các gradient được thu thập trong đường chuyền ngược.

## Full Implementation
Chúng tôi xác định `train_loop` lặp qua mã tối ưu hóa của chúng tôi và `test_loop` đánh giá hiệu suất của mô hình dựa trên dữ liệu thử nghiệm của chúng tôi.

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Chúng tôi khởi tạo hàm mất mát và trình tối ưu hóa, và chuyển nó vào `train_loop` và `test_loop`. Vui lòng tăng số lượng kỷ nguyên để theo dõi hiệu suất đang cải thiện của mô hình.

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298615  [    0/60000]
loss: 2.289773  [ 6400/60000]
loss: 2.286231  [12800/60000]
loss: 2.297304  [19200/60000]
loss: 2.272071  [25600/60000]
loss: 2.272357  [32000/60000]
loss: 2.273503  [38400/60000]
loss: 2.256310  [44800/60000]
loss: 2.260671  [51200/60000]
loss: 2.268083  [57600/60000]
Test Error: 
 Accuracy: 28.7%, Avg loss: 0.035359 

Epoch 2
-------------------------------
loss: 2.229062  [    0/60000]
loss: 2.226611  [ 6400/60000]
loss: 2.211932  [12800/60000]
loss: 2.266988  [19200/60000]
loss: 2.197991  [25600/60000]
loss: 2.183466  [32000/60000]
loss: 2.204887  [38400/60000]
loss: 2.151039  [44800/60000]
loss: 2.185158  [51200/60000]
loss: 2.209997  [57600/60000]
Test Error: 
 Accuracy: 33.7%, Avg loss: 0.034137 

Epoch 3
-------------------------------
loss: 2.120913  [    0/60000]
loss: 2.132128  [ 6400/60000]
loss: 2.098882  [12800/60000]
loss: 2.221612  [19200/60000]
loss: 2.081829  [25600/60000]
loss: 2.053265  [32000/600

## Further Reading
- [Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [`torch.optim`](https://pytorch.org/docs/stable/optim.html)
- [Warmstart Training a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)